# YOHO training

In [8]:
# Import used libraries

import pandas as pd

print("Pandas version: ", pd.__version__)

from utils import AudioFile, YOHODataset, YOHODataGenerator

Pandas version:  2.1.2


## Data generator

In [11]:
urbansed_df = YOHODataset(
    audios=[
        AudioFile(filepath=file.filepath, labels=file.events)
        for _, file in pd.read_csv("./data/urbansed.csv").iterrows()
    ]
)

print("x_0: ", urbansed_df[0][0])
print("y_0: ", urbansed_df[0][1])

x_0:  tensor([[[-0.0329,  0.4425,  0.5799,  ...,  1.3928, -0.0962, -0.2009],
         [-0.0348,  0.4374,  0.5723,  ...,  1.3889, -0.0655, -0.2012],
         [-0.0381,  0.4289,  0.5596,  ...,  1.3824, -0.0217, -0.2018],
         ...,
         [-1.1914, -1.4997, -1.3121,  ..., -1.9538, -1.2554, -2.2680],
         [-1.2754, -1.5072, -1.2038,  ..., -1.7031, -1.2814, -2.9766],
         [-1.3882, -1.6844, -1.2313,  ..., -1.4122, -1.3381, -2.5239]]])
y_0:  [('noise', 0, 10.0), ('jackhammer', 3.1254216166103967, 4.081540068079489), ('dog_bark', 4.272822510637965, 5.995148792556066), ('drilling', 4.644565713394142, 7.674565713394141)]


In [ ]:
urbansed_df[0][0].shape

In [ ]:
train_dataloader = YOHODataGenerator(urbansed_df, batch_size=1, shuffle=True)

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
train_features, train_labels